# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902023


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:01<00:26,  1.10it/s]

Rendering models:  10%|▉         | 3/31 [00:03<00:28,  1.01s/it]

Rendering models:  13%|█▎        | 4/31 [00:03<00:20,  1.30it/s]

Rendering models:  16%|█▌        | 5/31 [00:03<00:15,  1.66it/s]

...

Rendering models:  84%|████████▍ | 26/31 [00:11<00:01,  4.91it/s]

Rendering models:  87%|████████▋ | 27/31 [00:11<00:00,  4.94it/s]

Rendering models:  90%|█████████ | 28/31 [00:11<00:00,  4.71it/s]

Rendering models: 100%|██████████| 31/31 [00:11<00:00,  5.97it/s]

equidad1                              0.000392
ElisabethB                            0.000325
not-logged-in-cae133427d99aac1e554    0.000295
klmasters                             0.000247
Tommy80452                            0.000712
Thedunkmasta                          0.000862
mooberrysauce                         0.000418
GoGetThemJustin                       0.005661
elicarson                             0.002798
not-logged-in-314d48142a3a122ba666    0.001364
crush202020                           0.000286
databanana                            0.000246
gabrielweiss                          0.016942
acapirala                             0.000310
Lavadude                              0.007976
layne.woodward2701                    0.000311
not-logged-in-ee31e7d4412be094dae3    0.101049
not-logged-in-a018d7fdde63e8f6c885    0.000291
barrowwright                          0.000225
simon_harding                         0.001507
jmacd297                              0.001122
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())